In [1]:
from selenium import webdriver
from pyvirtualdisplay import Display
import gnp
import codecs
import json
from boto.s3.connection import S3Connection
from boto.s3.key import Key
import boto
import sys, os
import pandas as pd
from TwitterSearch import *
from sunlight import congress
from bs4 import BeautifulSoup
import urllib2
import itertools
import datetime
import configparser
import time
from pattern.en import parsetree
from pattern.en import mood
import dateutil.parser as dparser
import sunlight
from sunlight import congress
import re
from postal.parser import parse_address
import csv
import selenium
from bs4 import BeautifulSoup as soupy
import tweepy
import codecs
import io
from __future__ import unicode_literals

import requests
import json
import time
import codecs
import sys

In [2]:
# issues = ["civil rights",
# "women's rights","planned parenthood",
# "black lives matter","african american rights","black rights",
# "immigration","immigrants","undocumented","border wall","refugees","refugee rights",
# "gun control",
# "public education",
# "LGBTQ rights","LGBTQ","LGBT",
# "voting rights","redistricting","voter id laws","voting access","voter access"
#          ]


action_categories = {'donation': ["donation","donate","financial support","contribute","contribution"],
                     'protest': ["protest","rally","demonstration"],
                     'indivisible': ["meetup","congregate","gathering","indivisible"],
                     'boycott': ["boycott"],
                     'advocate': ["contact your representative","contact your senator","petition",
                                 "call your representative","call your senator",
                                 "email your representative","email your senator"],
                     'vote': ["vote","cast your ballot"],
                     'town hall': ["town hall","open office","town meeting","townhall",'virtual townhall','listening session']
                          }

issue_types = { 'immigrant rights' : ['immigrant rights',"immigrants' rights",'refugee rights', 'travel ban',
                                        'border wall','refugees','asylum','immigration reform',
                                        'immigrant advocacy','migrant rights','undocumented'],
               "women's rights" : ["women's rights","women's rights",'womens rights','gender equality'
                                  'girl power',"international women's day","war on women","planned parenthood"],
               "civil rights" : ["racial equality","black lives matter","african american rights","civil rights",
                                "black power","jim crow"],
               "LGTBQ rights" : ["marriage equality","transgender rights","equality act","lesbian rights",'gay rights',
                                'bisexual rights'],
               "voting rights": ["redistricting","gerrymandering","redistrict","gerrymander","voter id","voting access",
                                "voter access","voter suppression"],
               "worker rights": ['worker rights','right to work','minimum wage']
                      }

# organization_lists


In [18]:
config = configparser.ConfigParser()
config.read(os.path.join(os.pardir, 'config', 'ross.ini'))

consumer_key = config['twitter.api']['consumer_key']
consumer_secret = config['twitter.api']['consumer_secret']
access_token = config['twitter.api']['access_token']
access_token_secret = config['twitter.api']['access_token_secret']

AWS_ACCESS_KEY_ID = config['aws.creds']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = config['aws.creds']['AWS_SECRET_ACCESS_KEY']

meetup_api_key = config['meetup.api']['api_key']

In [4]:
legislators = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'legislators.csv'))

mask = (legislators.in_office == 1)

legislators = legislators.loc[mask]

legislators['title_name'] = legislators['title'].str.decode('utf-8','ignore') + ' ' + legislators['firstname'].str.decode('utf-8','ignore') + ' ' + legislators['lastname'].str.decode('utf-8','ignore')

print legislators.columns
print
print list(legislators['title_name'])[:10]
legislator_list = list(legislators['title_name'])

Index([            u'title',        u'full_title',         u'firstname',
              u'middlename',          u'lastname',       u'name_suffix',
                u'nickname',             u'party',             u'state',
                u'district',         u'in_office',            u'gender',
                   u'phone',               u'fax',           u'website',
                 u'webform',   u'congress_office',       u'bioguide_id',
            u'votesmart_id',            u'fec_id',       u'govtrack_id',
                  u'crp_id',        u'twitter_id', u'congresspedia_url',
             u'youtube_url',       u'facebook_id',      u'official_rss',
            u'senate_class',         u'birthdate',          u'oc_email',
              u'title_name'],
      dtype='object')

[u'Rep Robert Aderholt', u'Sen Lamar Alexander', u'Rep Justin Amash', u'Rep Mark Amodei', u'Rep Alma Adams', u'Rep Pete Aguilar', u'Rep Rick Allen', u'Rep Ralph Abraham', u'Rep Jodey Arrington', u'Rep Joe Barton']


In [5]:
immigration_orgs = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'immigration_orgs.csv'))
civil_rights_orgs = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'civil_rights_orgs.csv'))
lgbtq_orgs = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'lgbtq_orgs.csv'))

print lgbtq_orgs[:10]

# for org in immigration_orgs['org_name']:
#   print org


   Unnamed: 0                              org_name
0           0                       Accord Alliance
1           1                                ACT UP
2           2         Advocates for Informed Choice
3           3    Affirmation: Gay & Lesbian Mormons
4           4                      Against Equality
5           5  American Foundation for Equal Rights
6           6    American Veterans for Equal Rights
7           7                          Athlete Ally
8           8                        Atticus Circle
9           9                            Bash Back!


In [6]:
cities = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'Top5000Population.csv'))
cities['full_city'] = '\"' + cities['city'].str.rstrip() + ', ' + cities['state'] + '\"'
print cities[:10]
city_list = list(cities['city'].str.rstrip())
state_list = list(cities['state'].str.rstrip())
                 
city_state_tuples = zip(city_list, state_list)
print city_list[:10]
print city_state_tuples[:10]
# print city_list.index('Jersey City, NJ')

            city state population           full_city
0      New York     NY  8,363,710      "New York, NY"
1   Los Angeles     CA  3,833,995   "Los Angeles, CA"
2       Chicago     IL  2,853,114       "Chicago, IL"
3       Houston     TX  2,242,193       "Houston, TX"
4       Phoenix     AZ  1,567,924       "Phoenix, AZ"
5  Philadelphia     PA  1,447,395  "Philadelphia, PA"
6   San Antonio     TX  1,351,305   "San Antonio, TX"
7        Dallas     TX  1,279,910        "Dallas, TX"
8     San Diego     CA  1,279,329     "San Diego, CA"
9      San Jose     CA    948,279      "San Jose, CA"
['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 'Philadelphia', 'San Antonio', 'Dallas', 'San Diego', 'San Jose']
[('New York', 'NY'), ('Los Angeles', 'CA'), ('Chicago', 'IL'), ('Houston', 'TX'), ('Phoenix', 'AZ'), ('Philadelphia', 'PA'), ('San Antonio', 'TX'), ('Dallas', 'TX'), ('San Diego', 'CA'), ('San Jose', 'CA')]


In [7]:
keyword_searches = []
### Come up wiht long list of queries
for issue in issue_types.keys():
  for issue_keyword in issue_types[issue]:
    for category in action_categories.keys():
      for action_keyword in action_categories[category]: 
        for combination in itertools.product([issue_keyword],city_list):
          keyword_searches.append(' '.join(list(combination)))
      
for issue in issue_types.keys():
  for issue_keyword in issue_types[issue]:
    for category in action_categories.keys():
      for action_keyword in action_categories[category]:
        combination = [action_keyword,issue_keyword]
        keyword_searches.append(' '.join(combination))

for org in immigration_orgs:
  for category in action_categories.keys():
    for action_keyword in action_categories[category]: 
      combination = [action_keyword,org]
      keyword_searches.append(' '.join(combination))

for org in civil_rights_orgs:
  for category in action_categories.keys():
    for action_keyword in action_categories[category]: 
      combination = [action_keyword,org]
      keyword_searches.append(' '.join(combination))
          
for org in lgbtq_orgs:
  for category in action_categories.keys():
    for action_keyword in action_categories[category]: 
      combination = [action_keyword,org]
      keyword_searches.append(' '.join(combination))
          
for leg in legislator_list:
  for category in action_categories.keys():
    for category_keyword in action_categories[category]:
      combination = [leg,category_keyword]
      keyword_searches.append(' '.join(combination))

print len(keyword_searches)
print keyword_searches[-20:]

5756520
[u'Rep Ryan Zinke call your senator', u'Rep Ryan Zinke email your representative', u'Rep Ryan Zinke email your senator', u'Rep Ryan Zinke town hall', u'Rep Ryan Zinke open office', u'Rep Ryan Zinke town meeting', u'Rep Ryan Zinke townhall', u'Rep Ryan Zinke virtual townhall', u'Rep Ryan Zinke listening session', u'Rep Ryan Zinke protest', u'Rep Ryan Zinke rally', u'Rep Ryan Zinke demonstration', u'Rep Ryan Zinke boycott', u'Rep Ryan Zinke donation', u'Rep Ryan Zinke donate', u'Rep Ryan Zinke financial support', u'Rep Ryan Zinke contribute', u'Rep Ryan Zinke contribution', u'Rep Ryan Zinke vote', u'Rep Ryan Zinke cast your ballot']


In [46]:
today = datetime.datetime.today()
UTF8Writer = codecs.getwriter('utf8')
sys.stdout = UTF8Writer(sys.stdout)

def main_group(keyword,city_state):
  cities = city_state
  api_key= meetup_api_key
  for (city, state) in cities:
      per_page = 200
      results_we_got = per_page
      offset = 0
      while (results_we_got == per_page):
          # Meetup.com documentation here: http://www.meetup.com/meetup_api/docs/2/groups/
          response=get_results({"sign":"true","country":"US", "city":city, "state":state, "radius": 10,"text":keyword, "key":api_key, "page":per_page, "offset":offset })
          time.sleep(1)
          offset += 1
          for i,group in enumerate(response):
              category = ""
              if "category" in group:
                  category = group['category']['name']
              with open(os.path.join(os.pardir, 'data/documents', 'meetup_group_data_' + today.strftime("%Y%m%d%H") + '.txt'), 'a+') as outfile:
                json.dump(group,outfile)
                outfile.write(',')
                outfile.write('\n')

      time.sleep(1)



def get_results(params):

  request = requests.get("http://api.meetup.com/find/groups",params=params)
  data = request.json()

  return data

city_tuples = city_state_tuples[7:9]
main_group('protest',city_tuples)

In [53]:
today = datetime.datetime.today()
UTF8Writer = codecs.getwriter('utf8')
sys.stdout = UTF8Writer(sys.stdout)

def main_event(keyword,city_state):
  cities = city_state
  api_key= meetup_api_key
  for (city, state) in cities:
      per_page = 200
      results_we_got = per_page
      offset = 0
      while (results_we_got == per_page):
          # Meetup.com documentation here: http://www.meetup.com/meetup_api/docs/2/groups/
          response=get_results({"sign":"true","country":"US", "city":city, "state":state, "radius": 100,"text":keyword, "key":api_key, "page":per_page, "offset":offset })
          time.sleep(1)
          offset += 1
          for i,event in enumerate(response):
              category = ""
              if "category" in event:
                  category = group['category']['name']
              with open(os.path.join(os.pardir, 'data/documents', 'meetup_event_data_' + today.strftime("%Y%m%d%H") + '.txt'), 'a+') as outfile:
                json.dump(event,outfile)
                outfile.write(',')
                outfile.write('\n')

      time.sleep(1)



def get_results(params):

  request = requests.get("http://api.meetup.com/find/events",params=params)
  data = request.json()

  return data

city_tuples = city_state_tuples[13:15]
main_event('protest',city_tuples)

KeyboardInterrupt: 